In [1]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [2]:
news_content = "Queue는 한쪽 끝(rear)에서는 enqueue, 또 다른 끝(front)에서는 dequeue 연산을 하는 유한 순서리스트이다.Queue는 FIFO(First in First out) 리스트이다.front는 dequeue할 위치를 기록하고 rear은 enqueue할 위치를 기록한다.front와 rear가 같으면 큐는 비었다는 의미와 같다.rear는 입력할 데이터의 인덱스를 가리키기에 큐의 마지막 인덱스라면 더이상 원소를 집어넣을 수 없기에 에러를 발생시킨다. 그렇지 않다면 그 다음 인덱스에 원소를 넣는다.큐가 비었다면(rear == front) 빼낼 원소가 없기에 에러를 발생시키고 deque할 원소를 기록할 front를 1 더해준다.deque 연산과 비슷한 방식이다.peek: front가 가리키는 원소를 리턴한다.delete: front가 가리키는 원소를 삭제한다.(front + 1)순차표현의 문제점: 크기가 8인 배열에서 rear = 7, front = 3일때 0,1,2 인덱스는 비었지만 사용하지 못한다.순차표현의 문제를 해결하기 위해 원형 큐 고안rear를 하나 증가시켰을 때 rear = front 라면 가득찼다는 의미와 같다.원형 큐는 순차표현 큐에 비해 빈공간을 남기지 않아 메모리 공간을 잘 활용하지만 배열로 구현되기에 큐의 크기는 제한된다는 단점이 있다.한정된 크기를 개선하기 위해 연결리스트 큐 고안.연결리스트 표현으로 바뀌었지만 내부 동작은 순차표현 큐와 비슷하다."

In [3]:
kkma = Kkma()

In [4]:
def text2sentences(text):
    sentences = kkma.sentences(text)
    for idx in range(0,len(sentences)):
        if len(sentences[idx]) <= 10:
            sentences[idx-1] +=(' '+ sentences[idx])
            sentences[idx] = ''
    
    return sentences

In [5]:
sentences = text2sentences(news_content)

In [19]:
twitter = Twitter()
from konlpy.tag import Okt

okt = Okt()

C:\Users\SAMSUNG\anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [20]:
# nouns = okt.morphs(sentences)

In [71]:
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if len(noun)>1]))
                                
            
    return nouns

<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_23144\418709231.py:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [72]:
nouns = get_nouns(sentences)

In [73]:
print(sentences[0])

Queue는 한쪽 끝 (rear )에서는 enqueue, 또 다른 끝 (front )에서는 dequeue 연산을 하는 유한 순서 리스트이다.Queue


In [74]:
print(nouns[0])

한쪽 다른 연산 유한 순서 리스트


In [75]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence =[]

In [76]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

sent_graph = build_sent_graph(nouns)

In [77]:
sent_graph

array([[1.        , 0.33318459, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.24506429, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.13771091],
       [0.33318459, 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.41331715],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.25684549, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.27769034, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.3745748 , 0.34813357, 0.        , 0.11350255, 0.11350255,
        0.17829628, 0.08001045, 0.        , 0.        ],
       [0.        , 0.        , 0. 

In [78]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis =0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T,cnt_vec_mat),{vocab[word] :word for word in vocab}
    words_graph , idx2word = build_words_graph(nouns)
                 

In [79]:
words_graph = build_words_graph(nouns)

In [80]:
words_graph

(array([[1.        , 0.        , 0.70710678, ..., 0.        , 1.        ,
         0.        ],
        [0.        , 1.        , 0.70710678, ..., 0.        , 0.        ,
         0.        ],
        [0.70710678, 0.70710678, 1.        , ..., 0.        , 0.70710678,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
         0.        ],
        [1.        , 0.        , 0.70710678, ..., 0.        , 1.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.        ]]),
 {44: '한쪽',
  7: '다른',
  29: '연산',
  34: '유한',
  25: '순서',
  13: '리스트',
  32: '위치',
  5: '기록',
  35: '의미',
  38: '입력',
  10: '데이터',
  37: '인덱스',
  15: '마지막',
  12: '라면',
  36: '이상',
  30: '원소',
  27: '에러',
  19: '발생',
  8: '다음',
  22: '빼낼',
  20: '방식',
  14: '리턴',
  24: '삭제',
  26: '순차',
  42: '표현',
  18: '문제점',
  41: '크기',
  21: '배열',
  23: '사용',
  17: '문제',
  45: '해결',
  33: '위해',
  31: '원형',
  2: '고안',


In [81]:
def get_ranks(graph, d =0.85):   #d =단어사이간 유사한것과 따라갈 확률
    A=graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id,id]=0
        link_sum = np.sum(A[:,id])
        if link_sum != 0:
            A[:,id] /= link_sum
        A[:,id] *= -d
        A[id,id]= 1
        
    B = (1-d) * np.ones((matrix_size,1))
    
    ranks = np.linalg.solve(A,B)
    return {idx: r[0] for idx , r in enumerate(ranks)}

In [82]:
sent_rank_idx = get_ranks(sent_graph)


In [83]:
sent_rank_idx

{0: 1.0631930798658464,
 1: 1.000421817470008,
 2: 0.3750676041031084,
 3: 0.45458621541044386,
 4: 1.2303589706085738,
 5: 1.3042152564134653,
 6: 1.5285045955109324,
 7: 0.4593303058150262,
 8: 0.9744599285469528,
 9: 0.9744599285469527,
 10: 1.1992349442792072,
 11: 1.1755413991797288,
 12: 0.7784837892601485,
 13: 1.4821421649896036}

In [84]:
sorted_sent_rank_idx = sorted(sent_rank_idx , key = lambda k : sent_rank_idx[k] , reverse=True)

In [85]:
sorted_sent_rank_idx

[6, 13, 5, 4, 10, 11, 0, 1, 8, 9, 12, 7, 3, 2]

In [86]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx , key=lambda k : word_rank_idx[k], reverse = True)

AttributeError: 'tuple' object has no attribute 'shape'

In [90]:
def summarize(sent_num = 2):
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
    index.sort()
    
    for idx in index:
        summary.append(sentences[idx])
        
    for text in summary:
        print(text)
        print("\n")

In [91]:
summarize()

큐가 비었다면 (rear == front) 빼낼 원소가 없기에 에러를 발생시키고 deque 할 원소를 기록할 front를 1 더해 준다.deque


한 정된 크기를 개선하기 위해 연결 리스트 큐 고안. 연결 리스트 표현으로 바뀌었지만 내부 동작은 순차 표현 큐와 비슷하다.


